In [17]:
import json
import re
import os
from collections import Counter
from tqdm import tqdm

rst_dir = '/home/ubuntu/MMSci/mmsci-exps/eval/output/image_caption_matching'

cot = False
setting = 3
k = 5
rst_dir = os.path.join(rst_dir, "w_cot" if cot else "wo_cot", f"setting-{setting}", f"k_{k}")

In [2]:
def extract_ABCD_colon(input_string):
    input_string = input_string.strip().replace("\n", " ")
    pattern = r'(^| )([A-D]):'
    match = re.search(pattern, input_string)
    if match:
        return match.group(2)
    return None

In [7]:
def parse_prediction(str):
    
    if "[/INST]" in str:
        str = str.split("[/INST]")[1].strip()

    if extract_ABCD_colon(str) in ["A", "B", "C", "D"]:
        return extract_ABCD_colon(str).lower()
    
    choice_list = ['a', 'b', 'c', 'd']
    # remove all punctuations
    str = re.sub(r'[^\w\s]', '', str).lower()
    # replace '\n' with ' '
    str = str.replace('\n', ' ')
    # remove duplicated spaces
    str = re.sub(' +', ' ', str)
    if len(str) == 0:
        return 'wrong answer'
    # only keep the first 10 words
    str = ' '.join(str.split()[:10])

    if str.split()[0] in choice_list:  # first token is a/b/c/d
        return str.split()[0]
    for choice in choice_list:
        if str.endswith(f'is {choice}'):
            return choice
        if f'is {choice} ' in str:
            return choice
    return 'wrong answer'

In [8]:
test_list = [
    'A',
    '(a)',
    '(e)',
    'is describing',
    'bla bla bla is b',
    "The caption describing Figure (c) is:\n\nA",
    "The content in Figure (b) describes the temperature derivative of the resistivity, indicating aT-linear behaviour at high temperatures at \u03b5=\u22120.3%. The figure also shows the fit-free analysis of theT4/3power-law behaviour for \u03b5=\u22120.3%, theT5/3power-law behaviour for \u03b5=\u22122.9%, and the upper and lower temperature limits where the resistivity crosses over to a linearTbehaviour. The figure also shows the thermal hysteresis coercivity versus strain and q-scans across the E\u2032-type AFM reflection intensity at k=(1/4,1/4,1/4) observed by resonant X-ray diffraction at the Ni L3-edge at 12\u2009K and 140\u2009K. Therefore, the answer is C: The thermal hysteresis coercivity versus strain."
]
for test_str in test_list:
    print(test_str, '\n', parse_prediction(test_str), '\n', '-'*3)

A 
 a 
 ---
(a) 
 a 
 ---
(e) 
 wrong answer 
 ---
is describing 
 wrong answer 
 ---
bla bla bla is b 
 b 
 ---
The caption describing Figure (c) is:

A 
 a 
 ---
The content in Figure (b) describes the temperature derivative of the resistivity, indicating aT-linear behaviour at high temperatures at ε=−0.3%. The figure also shows the fit-free analysis of theT4/3power-law behaviour for ε=−0.3%, theT5/3power-law behaviour for ε=−2.9%, and the upper and lower temperature limits where the resistivity crosses over to a linearTbehaviour. The figure also shows the thermal hysteresis coercivity versus strain and q-scans across the E′-type AFM reflection intensity at k=(1/4,1/4,1/4) observed by resonant X-ray diffraction at the Ni L3-edge at 12 K and 140 K. Therefore, the answer is C: The thermal hysteresis coercivity versus strain. 
 c 
 ---


In [9]:
def most_frequent_item(lst):
    # Create a dictionary to store the count of each item
    counts = {}
    # Iterate over the list and count occurrences
    for item in lst:
        if item in counts:
            counts[item] += 1
        else:
            counts[item] = 1
    
    # Find the item with the maximum count
    max_count = 0
    most_frequent = None
    for item in lst:
        if counts[item] > max_count:
            max_count = counts[item]
            most_frequent = item
    
    return most_frequent

# Example usage
input1 = ["A", "D", "A", "A", "B"]
input2 = ["A", "D", "A", "D", "B"]
print(most_frequent_item(input1))  # Output: "A"
print(most_frequent_item(input2))  # Output: "A"

A
A


In [20]:

for file in os.listdir(rst_dir):
    rst_list = json.load(open(os.path.join(rst_dir, file)))
    num_total = len(rst_list)
    num_correct = [0, 0, 0] # 1, 3, 5
    valid = [0, 0, 0] # 1, 3, 5
    for item in tqdm(rst_list):
        # counter = Counter
        gt = item['answer'].lower()
        preds = [] if 'prediction' not in item else item['prediction'] # only select the top_k pred for evaluation
        preds = [pred["extracted_answer"].lower() if "w-ans" in file else parse_prediction(pred["answer"]) for pred in preds]
        for idx, top_k in enumerate([1, 3, 5]):
            top_preds = preds[:top_k]
            if most_frequent_item(top_preds) == gt:
                num_correct[idx] += 1
            if top_preds:
                valid[idx] += 1
    for idx, top_k in enumerate([1, 3, 5]):
        print(f'{file}: {top_k}: {num_correct[idx]}/{num_total} = {num_correct[idx]*100/num_total:.2f}%')
        print(f'{file}: {top_k}: {num_correct[idx]}/{valid[idx]} = {num_correct[idx]*100/valid[idx]:.2f}%')
        print()

  0%|          | 0/1119 [00:00<?, ?it/s]

100%|██████████| 1119/1119 [00:00<00:00, 13935.23it/s]


llava-next-mistral.json: 1: 265/1119 = 23.68%
llava-next-mistral.json: 1: 265/1119 = 23.68%

llava-next-mistral.json: 3: 259/1119 = 23.15%
llava-next-mistral.json: 3: 259/1119 = 23.15%

llava-next-mistral.json: 5: 243/1119 = 21.72%
llava-next-mistral.json: 5: 243/1119 = 21.72%



100%|██████████| 1119/1119 [00:00<00:00, 17334.40it/s]


llava-next.json: 1: 244/1119 = 21.81%
llava-next.json: 1: 244/1119 = 21.81%

llava-next.json: 3: 249/1119 = 22.25%
llava-next.json: 3: 249/1119 = 22.25%

llava-next.json: 5: 255/1119 = 22.79%
llava-next.json: 5: 255/1119 = 22.79%



...

gpt-4-turbo.json: 1: 664/1117 = 59.44%
gpt-4-turbo.json: 1: 664/1117 = 59.44%

gpt-4-turbo.json: 3: 718/1117 = 64.28%
gpt-4-turbo.json: 3: 718/1117 = 64.28%

gpt-4-turbo.json: 5: 743/1117 = 66.52%
gpt-4-turbo.json: 5: 743/1117 = 66.52%



100%|██████████| 1119/1119 [00:00<00:00, 69812.52it/s]


llava-next-mmsci.json: 1: 850/1119 = 75.96%
llava-next-mmsci.json: 1: 850/1119 = 75.96%

llava-next-mmsci.json: 3: 861/1119 = 76.94%
llava-next-mmsci.json: 3: 861/1119 = 76.94%

llava-next-mmsci.json: 5: 864/1119 = 77.21%
llava-next-mmsci.json: 5: 864/1119 = 77.21%



100%|██████████| 1117/1117 [00:00<00:00, 26128.14it/s]

gpt-4o.json: 1: 967/1117 = 86.57%
gpt-4o.json: 1: 967/1117 = 86.57%

gpt-4o.json: 3: 977/1117 = 87.47%
gpt-4o.json: 3: 977/1117 = 87.47%

gpt-4o.json: 5: 983/1117 = 88.00%
gpt-4o.json: 5: 983/1117 = 88.00%

